# Using Featuretools to Predict Missed Appointments
In this notebook, [Featuretools](https://github.com/Featuretools/featuretools) to automatically generate features relating to when patients don't show up for doctor appointments. We follow the approach in the most popular [kernel]() to demonstrate the ways in which Featuretools simplifies and extends common data science operations. An advantage of using automated feature extraction is that it allows us to flip the standard data science process: we can explore our data *after* building interesting features.

To get started, download the [Kaggle](https://www.kaggle.com/joniarroba/noshowappointments/data) data on appointment noshows and store it in a `data` folder in this repository.


In [1]:
import numpy as np
import pandas as pd
import featuretools as ft
ft.__version__

'0.1.18'

# Structuring the Data
The key to the whole game is setting up a dataset in a way that accurately represents what we are interested in learning. To start, we'll load the data into a `pandas.DataFrame` and do some minor modifications so that I can more easily remember the column names.

In [2]:
data = pd.read_csv("data/KaggleV2-May-2016.csv")
data.index = data['AppointmentID']
data.rename(columns = {'Hipertension': 'Hypertension',
                       'Handcap': 'Handicap',
                       'PatientId': 'PatientID',
                       'Neighbourhood': 'Neighborhood',
                       'No-show': 'Label'}, inplace = True)
data['Label'] = data['Label'].map({'No': 0, 'Yes': 1})
#data['ScheduledDay'] = pd.to_datetime(data['ScheduledDay'])
#data['AppointmentDay'] = pd.to_datetime(data['AppointmentDay'])
data.head(2)



,PatientID,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighborhood,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMS_received,Label
AppointmentID,,,,,,,,,,,,,,
5642903,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,0
5642503,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,0


Next, we set up an [EntitySet](https://docs.featuretools.com/loading_data/using_entitysets.html). An EntitySet is a way of storing data, data metadata, and relationships which makes it possible to automatically generate features. 

Even though the types of columns have been set when we loaded our data into a `pandas.DataFrame`, we might want to specify a different type for a column explicitly. A standard example comes up in this dataset: `Age` is a Numeric type (every element is a number) but it's more accurate to think of it as an ordered categorical type of information (Ordinal). Our other columns are also Numeric, but would be more accurately described as Boolean (0 or 1).

In [3]:
import featuretools.variable_types as vtypes

# Give featuretools column metadata
variable_types = {'Gender': vtypes.Categorical,
                  'Age': vtypes.Ordinal,
                  'Scholarship': vtypes.Boolean,
                  'Hypertension': vtypes.Boolean,
                  'Diabetes': vtypes.Boolean,
                  'Alcoholism': vtypes.Boolean,
                  'Handicap': vtypes.Boolean,
                  'Label': vtypes.Boolean,
                  'SMS_received': vtypes.Boolean}

We call our EntitySet "Appointments" and build it from the ground up. First, we use `entity_from_dataframe` to turn our data into an entity and to apply the variable types. We also set a time index to attatch a particular datetime to every row. However, in this dataset, not all columns occur at the same time.

In the provided data we are given both the time an appointment is set (`ScheduledDay`), and the actual time of the appointment (`AppointmentDay`). Most notably, we won't know whether or not a person shows up to their appointment when they schedule it. Accounting for that difference by assigning `Labels` a `secondary_time_index` lets us lean on Deep Feature Synthesis to not leak Label information into any predictions we might make.

In [4]:
data['AppointmentTime'] = data['AppointmentDay']

# Create an `EntitySet` named `appointment_data`
es = ft.EntitySet('Appointments')

# Make an entity named 'appointments' which stores dataset metadata with the dataframe
es = es.entity_from_dataframe(entity_id="appointments",
                              dataframe=data,
                              index='AppointmentID',
                              time_index='ScheduledDay',
                              secondary_time_index={'AppointmentDay': ['Label', 'SMS_received']},
                              variable_types=variable_types)

Next, we choose some interesting variables to group by and make additional entities to represent those concepts. We can theorize that people might be more or less likely to show up depending on their age. By `normalizing` our entity, we create a structure which groups together events where the `Age` column is the same. We'll do the same thing with the gender of the patient and `PatientID`. It's easy to suspect that if a particular patient has cancelled in the past, that might have an impact on if they will in the future.

Notice that along with new entities the relationships are also automatically generated.

In [5]:
es.normalize_entity('appointments', 'patients', 'PatientID', 
                    make_time_index=True)
es.normalize_entity('appointments', 'ages', 'Age',
                    make_time_index=False)
es.normalize_entity('appointments', 'genders', 'Gender',
                    make_time_index=False)

Entityset: Appointments
  Entities:
    appointments (shape = [110527, 15])
    patients (shape = [62299, 2])
    ages (shape = [104, 1])
    genders (shape = [2, 1])
  Relationships:
    appointments.PatientID -> patients.PatientID
    appointments.Age -> ages.Age
    appointments.Gender -> genders.Gender

# Generating Features with Deep Feature Synthesis
With our data nicely structued in an **EntitySet**, we can immediately build useful features. We can create features with built in primitives like `Hour`, which returns the hour of a datetime and `PercentTrue`, which will calculate how often a boolean value is true. As an example, a features like `ages.PERCENT_TRUE(Label)` will calculate the probability that a patient doesn't show up based on their age.

It is at this stage that our time indices come into play. Since we set the cutoff time to be the Appointment Day, Deep Feature Synthesis will only use data that was obtained prior to that cutoff time for each calculation. This gives us the flexibility to not care about which value in the time series we're trying to predict: our data will be valid at each point. Let's calculate features and look at the first 20 columns.

In [6]:
from featuretools.primitives import Weekday, Hour, Count, Day, NUnique, PercentTrue

cutoff_times = data[['AppointmentID', 'AppointmentDay']]
cutoff_times['AppointmentDay'] = pd.to_datetime(cutoff_times['AppointmentDay'])


fm, features = ft.dfs(entityset=es,
                      target_entity='appointments',
                      agg_primitives=[Count, NUnique, PercentTrue],
                      trans_primitives=[Weekday, Hour, Day],
                      drop_contains=['AppointmentDay'],
                      max_depth=3,
                      cutoff_time=cutoff_times,
                      features_only=False,
                      verbose=True)
fm.head(3).iloc[:,30:40]

Removed 12 features from drop_exact and drop_contains lists                    
Building features: 87 features [00:00, 3812.61 features/s, features excluded=6]
Elapsed: 01:35 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 27/27 cutoff times
Computed 69 features


,ages.NUM_UNIQUE(appointments.PatientID),ages.NUM_UNIQUE(appointments.Neighborhood),ages.NUM_UNIQUE(appointments.Gender),ages.PERCENT_TRUE(appointments.Scholarship),ages.PERCENT_TRUE(appointments.Hypertension),ages.PERCENT_TRUE(appointments.Diabetes),ages.PERCENT_TRUE(appointments.Alcoholism),ages.PERCENT_TRUE(appointments.Handicap),ages.PERCENT_TRUE(appointments.Label),ages.PERCENT_TRUE(appointments.SMS_received)
AppointmentID,,,,,,,,,,
5217179,60.0,32.0,2.0,0.0,0.539683,0.142857,0.000000,0.063492,0.0,0.0
5218520,69.0,34.0,2.0,0.0,0.688312,0.142857,0.000000,0.064935,0.0,0.0
5235449,143.0,43.0,2.0,0.0,0.604790,0.299401,0.017964,0.077844,0.0,0.0


In [7]:
fm[['Age', 'Gender', 'Neighborhood', 'ages.PERCENT_TRUE(appointments.Label)']].head(3)

,Age,Gender,Neighborhood,ages.PERCENT_TRUE(appointments.Label)
AppointmentID,,,,
5217179,84.0,M,SANTO ANDRÉ,0.0
5218520,83.0,F,REDENÇÃO,0.0
5235449,74.0,F,MONTE BELO,0.0


Some of these features are strings, which won't work with machine learning. We'll use the built in feature matrix encoder to make new columns which one-hot-encode 'Neighborhood' and 'Gender'.

In [8]:
enc_fm, _ = ft.synthesis.encode_features(fm, features, 
                                            top_n=5, 
                                            include_unknown=False, 
                                            to_encode=['Neighborhood', 'Gender'], 
                                            inplace=False, 
                                            verbose=True)
enc_fm.tail(5)

Encoding pass 2: 100%|██████████| 74/74 [00:00<00:00, 2387.03feature/s]


,PatientID,Neighborhood = JARDIM CAMBURI,Neighborhood = MARIA ORTIZ,Neighborhood = RESISTÊNCIA,Neighborhood = JARDIM DA PENHA,Neighborhood = ITARARÉ,Gender = F,Gender = M,Age,Scholarship,...,ages.NUM_UNIQUE(appointments.HOUR(ScheduledDay)),ages.NUM_UNIQUE(appointments.HOUR(AppointmentTime)),ages.NUM_UNIQUE(appointments.DAY(ScheduledDay)),ages.NUM_UNIQUE(appointments.DAY(AppointmentTime)),genders.NUM_UNIQUE(appointments.WEEKDAY(ScheduledDay)),genders.NUM_UNIQUE(appointments.WEEKDAY(AppointmentTime)),genders.NUM_UNIQUE(appointments.HOUR(ScheduledDay)),genders.NUM_UNIQUE(appointments.HOUR(AppointmentTime)),genders.NUM_UNIQUE(appointments.DAY(ScheduledDay)),genders.NUM_UNIQUE(appointments.DAY(AppointmentTime))
AppointmentID,,,,,,,,,,,,,,,,,,,,,
5790461,NaN,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5790464,NaN,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5790466,NaN,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5790481,NaN,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5790484,NaN,0,0,0,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 3: Predict

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

enc_fm = enc_fm.fillna(0)
labels = es['appointments'].df['Label']
X_train, X_test, y_train, y_test = train_test_split(enc_fm, labels, test_size=0.40)
clf = RandomForestClassifier(n_estimators=50)
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
print('AUC score of {:.2f}'.format(roc_auc_score(preds, y_test)))

AUC score of 0.55


In [10]:
feature_imps = [(imp, enc_fm.columns[i]) for i, imp in enumerate(clf.feature_importances_)]
feature_imps.sort()
feature_imps.reverse()
print('Feature Importances:')
for i, f in enumerate(feature_imps[0:5]):
    print('{}: {} [{:.3f}]'.format(i + 1, f[1], f[0]))


Feature Importances:
1: PatientID [0.085]
2: patients.DAY(first_appointments_time) [0.050]
3: DAY(ScheduledDay) [0.050]
4: HOUR(ScheduledDay) [0.049]
5: patients.HOUR(first_appointments_time) [0.049]


# Data exploration

In [11]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column

hist1 = data[['Age', 'PatientID']].groupby(['Age']).count()
hist2 = data[['Neighborhood', 'PatientID']].groupby(['Neighborhood']).count()

output_notebook()
plot1 = figure(plot_width=900, plot_height=300, title='Patients by Age')
plot1.vbar(x=hist1.index, 
           width=0.5, 
           bottom=0, 
           top=hist1.iloc[:,0], 
           color='purple', 
           alpha=.7)

plot2 = figure(x_range = list(hist2.index), plot_width=900, plot_height=500, title='Patients by Neighbourhood')
plot2.vbar(x=hist2.index, width=0.5, 
           bottom=0, 
           top=hist2.iloc[:,0], 
           alpha=.5, 
           color='red')
plot2.xaxis.major_label_orientation = 1.57


show(column(plot1, plot2))

Loading BokehJS ...

In [12]:
from bokeh.models import HoverTool

# prepare some data
x = fm[fm['DAY(AppointmentTime)']==8]['Age']
y = fm[fm['DAY(AppointmentTime)']==8]['ages.PERCENT_TRUE(appointments.Label)']
x, y = zip(*set(zip(x,y)))


hover = HoverTool(tooltips=[
    ("Prob", "$y{%0}"),
    ("Age", "$x{0}"),
])
# output to static HTML file
output_notebook()
# create a new plot with a title and axis labels
p = figure(title="Probability patient doesn't show up by age", 
           x_axis_label='Age', 
           y_axis_label='Probability of NoShow',
           tools=[hover])

# add a line renderer with legend and line thickness
p.scatter(x, y, alpha=.7, radius=1.5)

# show the results
show(p)

Loading BokehJS ...

In [17]:
from featuretools.primitives import make_trans_primitive
from featuretools.primitives import Weekday, Hour, Count, Day, NUnique, PercentTrue

cutoff_times = data[['AppointmentID', 'AppointmentDay']]
cutoff_times['AppointmentDay'] = pd.to_datetime(cutoff_times['AppointmentDay'])

def high_volume(index):
    A = pd.DataFrame(index, columns=['Name'])
    A['ones'] = 1
    gb = A.groupby(['Name']).count().to_dict()['ones']
    #import pdb; pdb.set_trace()
    output=[]

    for key in index:
        if gb[key] >= 5:
            output.append(1)
        else:
            output.append(0)
    return output
    
FreqFlier = make_trans_primitive(high_volume,
                                 input_types=[vtypes.Id],
                                 cls_attributes={'needs_all_values': True},
                                 return_type=vtypes.Boolean)
new_fm, new_features = ft.dfs(entityset=es,
                              target_entity='appointments',
                              agg_primitives=[Count, PercentTrue, NUnique],
                              drop_contains=['AppointmentDay'],
                              max_depth=3,
                              cutoff_time=cutoff_times,
                              trans_primitives=[FreqFlier, Weekday, Day, Hour],
                              features_only=False,
                              verbose=True)

Removed 12 features from drop_exact and drop_contains lists                    
Building features: 99 features [00:00, 3627.97 features/s, features excluded=6]
Elapsed: 01:53 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 27/27 cutoff times
Computed 81 features


In [18]:
new_fm.head(10)

,PatientID,Neighborhood,Gender,Age,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,HIGH_VOLUME(PatientID),...,ages.NUM_UNIQUE(appointments.HOUR(AppointmentTime)),genders.PERCENT_TRUE(appointments.HIGH_VOLUME(PatientID)),genders.PERCENT_TRUE(appointments.HIGH_VOLUME(Gender)),genders.PERCENT_TRUE(appointments.HIGH_VOLUME(Age)),genders.NUM_UNIQUE(appointments.WEEKDAY(ScheduledDay)),genders.NUM_UNIQUE(appointments.WEEKDAY(AppointmentTime)),genders.NUM_UNIQUE(appointments.DAY(ScheduledDay)),genders.NUM_UNIQUE(appointments.DAY(AppointmentTime)),genders.NUM_UNIQUE(appointments.HOUR(ScheduledDay)),genders.NUM_UNIQUE(appointments.HOUR(AppointmentTime))
AppointmentID,,,,,,,,,,,,,,,,,,,,,
5217179,1.423329e+12,SANTO ANDRÉ,M,84.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.008640,1.0,0.999882,5.0,5.0,31.0,23.0,15.0,1.0
5218520,4.616858e+12,REDENÇÃO,F,83.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.013899,1.0,0.999766,6.0,5.0,31.0,23.0,16.0,1.0
5235449,5.558963e+13,MONTE BELO,F,74.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.013899,1.0,0.999766,6.0,5.0,31.0,23.0,16.0,1.0
5235643,9.189694e+13,GURIGICA,F,70.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.013899,1.0,0.999766,6.0,5.0,31.0,23.0,16.0,1.0
5235655,1.534482e+12,JUCUTUQUARA,F,87.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.013899,1.0,0.999766,6.0,5.0,31.0,23.0,16.0,1.0
5236116,3.136481e+14,REDENÇÃO,M,71.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.008640,1.0,0.999882,5.0,5.0,31.0,23.0,15.0,1.0
5236380,1.596183e+14,PRAIA DO CANTO,F,88.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.013899,1.0,0.999766,6.0,5.0,31.0,23.0,16.0,1.0
5303666,9.646768e+13,RESISTÊNCIA,F,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.013899,1.0,0.999766,6.0,5.0,31.0,23.0,16.0,1.0
5304747,7.437646e+14,MARUÍPE,M,48.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.008640,1.0,0.999882,5.0,5.0,31.0,23.0,15.0,1.0


In [21]:
from bokeh.plotting import figure
from bokeh.colors.groups import purple
temp = fm[:10000]

colormap = {name: purple[i % 5] for i, name in enumerate(temp['Neighborhood'])}
colors = [colormap[x] for x in temp['Neighborhood']]
p = figure(title='Schedule frequency by day of month', tools = ['box_zoom', 'reset'])
p.scatter(x=temp['PatientID'], y=temp['patients.DAY(first_appointments_time)'], color=colors)

p.xaxis.axis_label = 'Patient ID'
p.yaxis.axis_label ='Day of Scheduled Month'
show(p)